# Exposing API data via Flask
> An example of API data transformation and serving.

- toc: true 
- badges: true
- comments: true
- categories: [python, api, webservice, flask]
- image: images/copied_from_nb/images/2021-06-08-exposing-api-data-via-flask-main.jpg

# About

We are going to pull data from a webservice, transform it and serve it back via a webservice.

![Photo by Andrea Davis](./images/2021-06-08-exposing-api-data-via-flask-main.jpg)

## The required libraries

In [34]:
import airbnb as ab
import pandas as pd
import locale as lo
import argparse as ap
import traceback as tb
import os as os
import functools as ft
from dateutil import parser as dtp

As we'll expose webservice endpoints, we should import the following `flask` elements.
> Those elements can be installed using `pip install flask flask-restful flask-wtf flask-cors`

In [9]:
import flask as fl
import flask_restful as flr
import flask_restful.reqparse as flr_r
import flask_cors as flc
import flask_wtf.csrf as flwc

## Get data from Airbnb API

> Let's pull `reviews` and `rating` information from **Airbnb** for a given `listing_id` (a property available to rent on Airbnb).

In [44]:
def get_reviews(
    listing_id, 
    locale='fr', 
    page_size=50, 
    keep_n_reviews=10, 
    min_rating=3,
    most_recent_first=True):

    api = ab.api.Api(randomize=True)
    api._session.headers['x-airbnb-locale'] = locale
    api._session.headers['accept-language'] = locale
    lo.setlocale(lo.LC_ALL, locale)
    
    reviews = []
    offset = 0
    reviews_count = 0
    has_more_reviews = True
    reviews_count = -1

    while has_more_reviews:
        page_reviews = api.get_reviews(
            listing_id=listing_id, 
            offset=offset, 
            limit=page_size)

        for review in page_reviews['reviews']:
            review['created_at__dt'] = dtp.parse(review['created_at'])
            reviews.append(review)

        reviews_count = page_reviews['metadata']['reviews_count']

        if len(page_reviews['reviews']) < page_size:
            has_more_reviews = False
        else:
            offset += page_size

    if min_rating is not None:
        reviews = [x for x in reviews if x['rating'] >= min_rating]
    
    def compare_reviews(left, right, most_recent_first=most_recent_first):
        if most_recent_first:
            if left['created_at__dt'] < right['created_at__dt']:
                return 1
            elif left['created_at__dt'] == right['created_at__dt']:
                return 0
            elif left['created_at__dt'] > right['created_at__dt']:
                return -1
            
    reviews = sorted(
        reviews, 
        key=ft.cmp_to_key(compare_reviews))
    
    if keep_n_reviews > 0:
        reviews = reviews[:keep_n_reviews]
        
    return reviews

In [45]:
def get_listing_details(
    listing_id, 
    locale='fr'):
    
    api = ab.api.Api(randomize=True)
    api._session.headers['x-airbnb-locale'] = locale
    api._session.headers['accept-language'] = locale
    lo.setlocale(lo.LC_ALL, locale)
    
    url_base = ab.api.API_URL
    
    r = api._session.get(
        '{0}/pdp_listing_details/{1}'.format(
            url_base,
            listing_id), 
        params={ '_format': 'for_rooms_show'})
    
    r.raise_for_status()
    
    return r.json()

In [49]:
def get_ratings(
    listing_id, 
    locale='fr',
    keep_n_reviews=10, 
    min_rating=3,
    most_recent_first=True):
    
    listing_details = get_listing_details(
        listing_id=listing_id, 
        locale=locale)
    
    reviews = get_reviews(
        listing_id=listing_id, 
        locale=locale, 
        keep_n_reviews=keep_n_reviews, 
        min_rating=min_rating,
        most_recent_first=most_recent_first)
    
    overall_rating_text = listing_details['pdp_listing_detail']['reviews_module']['localized_overall_rating']
    overall_rating_terms = listing_details['pdp_listing_detail']['reviews_module']['appreciation_tags']
    
    review_details = listing_details['pdp_listing_detail']['review_details_interface']

    overall_rating = lo.atof(overall_rating_text)
    review_count = review_details['review_count']
    review_summary = review_details['review_summary']
    
    for summary in review_summary:
        summary['rating'] = lo.atof(summary['localized_rating'])
        
    return {
        'overall_rating': overall_rating, 
        'review_count': review_count, 
        'categories': review_summary,
        'rating_terms': overall_rating_terms,
        'reviews': reviews
    }

We can get the individual `reviews` by calling the `get_reviews` function defiend above:

In [50]:
listing_id = 36902451

In [52]:
get_ratings(listing_id=listing_id, keep_n_reviews=10)

{'accept': 'application/json', 'accept-encoding': 'br, gzip, deflate', 'content-type': 'application/json', 'x-airbnb-api-key': '915pw2pnf4h1aiguhph5gc5b2', 'user-agent': 'Airbnb/17.13 iPhone/10.3.1 Type/Phone', 'x-airbnb-screensize': 'w=375.00;h=812.00', 'x-airbnb-carrier-name': 'T-Mobile', 'x-airbnb-network-type': 'wifi', 'x-airbnb-currency': 'USD', 'x-airbnb-locale': 'fr', 'x-airbnb-carrier-country': 'us', 'accept-language': 'fr', 'airbnb-device-id': '9a9092886fafc1b44f13c1a26cbb9ae0b1207e4c', 'x-airbnb-advertising-id': '09A2B1E9-278F-4E1C-9159-5659F82D890B'}


{'overall_rating': 4.97,
 'review_count': 31,
 'categories': [{'category': 'accuracy',
   'value': 10,
   'label': 'Précision',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'communication',
   'value': 10,
   'label': 'Communication',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'cleanliness',
   'value': 10,
   'label': 'Propreté',
   'localized_rating': '5,0',
   'percentage': 0.994,
   'rating': 5.0},
  {'category': 'location',
   'value': 10,
   'label': 'Emplacement',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'checkin',
   'value': 10,
   'label': 'Arrivée',
   'localized_rating': '5,0',
   'percentage': 1.0,
   'rating': 5.0},
  {'category': 'value',
   'value': 10,
   'label': 'Qualité-prix',
   'localized_rating': '5,0',
   'percentage': 0.994,
   'rating': 5.0}],
 'rating_terms': [{'localized_text': 'Propreté impeccable',
   'localized_count_string': '1

## Publishing data as a webservice

In [ ]:
class Info(flr.Resource):

    def __init__(self):
        super(Info, self).__init__()

    def get(self):
        return {
            'success': True,
            'version': '0.1.0',
            'status': 'ready'
        }

In [ ]:
class Hosting(flr.Resource):

    def __init__(self, **kwargs):

        print('__init__ for model "{0}"'.format(kwargs['model_id']))

        self.model = Statics.g_models[kwargs['model_id']]['model']
        self.reqparse = flr_r.RequestParser()

        for feature in self.model.input_features:
            self.reqparse.add_argument(
                feature.feature_name,
                type=str,
                location='json',
                required=True,
                help='"'+ feature.feature_name +'" is mandatory ({error_msg})')

        super(Hosting, self).__init__()

    def post(self):

        success = False
        error = None
        error_details = None
        result = None

        try:
            args = self.reqparse.parse_args()
            
            data = {}

            for feature in self.model.input_features:
                data[feature.feature_name] = args[feature.feature_name]

            result = self.model.predict(
                input=data)

            success = True

        except Exception as e:
            error_details = tb.format_exc()
            error = e.__str__()

        return {
            'success': success,
            'error': error,
            'error_details': error_details,
            'result': result
        }